# **Import Libraries**

In [ ]:
import numpy as np
import os
import tensorflow as tf
import pandas as pd
import glob
import math
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
from PIL import Image
from collections import Counter

# Train data preprocessing

In [ ]:
train_cultivar= pd.read_csv('/kaggle/input/sorghum-id-fgvc-9/train_cultivar_mapping.csv')
train_cultivar.dropna(inplace=True)
train_cultivar['cultivar']=train_cultivar['cultivar'].astype(str)
train_cultivar.head()

In [ ]:
labels=list(np.unique(train_cultivar['cultivar']))

In [ ]:
import seaborn as sns
a=pd.DataFrame({'cultivar':train_cultivar['cultivar']})
plt.figure(figsize=(30,10))
sns.histplot(a,x='cultivar')
plt.xticks(rotation=45)
plt.show()

In [ ]:
train_datagen=tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True, 
                                                              rotation_range=0.9,
                                                              brightness_range=(0.1,0.9))
train_data=train_datagen.flow_from_dataframe(train_cultivar, 
                                             directory="/kaggle/input/sorghum-id-fgvc-9/train_images/",
                                             color_mode='rgb', 
                                             batch_size=32, 
                                             x_col='image', 
                                             y_col='cultivar', 
                                             classes=labels, 
                                             class_mode='categorical', 
                                             target_size=(300, 300))

In [ ]:
len_traindata=len(train_cultivar)

len_traindata

# **EfficientnetB2(Base Model)**

In [ ]:
base_model= tf.keras.applications.efficientnet.EfficientNetB2(include_top=False, 
                                                              weights='imagenet', 
                                                              input_shape=(300,300,3),
                                                              pooling='avg')


In [ ]:
image_batch,label_image= next(iter(train_data))
feature_batch=base_model(image_batch)
feature_batch.shape

# **Design the Model**

In [ ]:
inputs=tf.keras.Input(shape=(300,300,3))
x=base_model(inputs, training=False)
x = tf.keras.layers.Dropout(0.5)(x)
outputs=tf.keras.layers.Dense(100, activation='softmax')(x)
model=tf.keras.Model(inputs, outputs)

# **Callbacks**

In [ ]:
class callback1(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs={}):
        if (logs.get('accuracy')>0.70):
            print('\nStopping training since train accuracy is greater than 70%')
            self.model.stop_training=True

In [ ]:
callback2=tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)

In [ ]:
class callback3(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epochs, logs={}):
        if (logs.get('accuracy')>0.97):
            print('\nStopping training since train accuracy is greater than 97%')
            self.model.stop_training=True

In [ ]:
callback4=tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3, restore_best_weights=True)

# **Cyclical Learning Rate**

In [ ]:

batch_size=32
if abs(len_traindata/batch_size-round(len_traindata/batch_size))>0.5:
    total_steps=round(len_traindata/batch_size)
else:
    total_steps=round(len_traindata/batch_size)+1
clr = tfa.optimizers.CyclicalLearningRate(initial_learning_rate=1e-4,
    maximal_learning_rate=1e-3,
    scale_fn=lambda x: 1/(2.**(x-1)),
    step_size=2 * total_steps
)


# **Compile the model**

In [ ]:
for layer in base_model.layers:
    layer.trainable=False

In [ ]:
model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

# **Fit the model and Visualize**

In [ ]:
tf.keras.utils.plot_model(model)

In [ ]:
history=model.fit(train_data, epochs=25, callbacks=[callback1(), callback2])

In [ ]:
pd.DataFrame(history.history).plot()
plt.ylabel('loss & accuracy')
plt.xlabel('epochs')

# **Fine Tuning**

In [ ]:
count=0
for i in base_model.layers:
    count+=1
count

In [ ]:
for layer1 in base_model.layers[280:]:
    layer1.trainable=True

In [ ]:
model.summary()

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001), metrics=['accuracy'])

In [ ]:
history_fine=model.fit(train_data, epochs=40,initial_epoch=history.epoch[-1],callbacks=[callback3(), callback4])

In [ ]:
pd.DataFrame(history_fine.history).plot()
plt.ylabel('loss & accuracy')
plt.xlabel('epochs')

# **Prepare Test Data**

In [ ]:
test_data=pd.read_csv('/kaggle/input/sorghum-id-fgvc-9/sample_submission.csv').drop('cultivar',axis=1)
test_data

In [ ]:
test_datagen=tf.keras.preprocessing.image.ImageDataGenerator()
test_dataset=test_datagen.flow_from_dataframe(test_data,directory="/kaggle/input/sorghum-id-fgvc-9/test/", x_col='filename',y_col=None, color_mode='rgb', target_size=(300,300), shuffle=False, class_mode=None)

# **Predict**

In [ ]:
predict1=model.predict(test_dataset)


In [ ]:
predict1.shape

In [ ]:
predict2=np.argmax(predict1, axis=-1)

In [ ]:
predictions=[]
for k in predict2:
    predictions.append(labels[k])
  


In [ ]:
final=pd.DataFrame({'filename': test_data.filename, 'cultivar': predictions})
final.to_csv('submission.csv', index=False)